from utility import *
from samplers import *
from pool import *

def runMe(sequence_length,num_rounds=3,affinity_max=1e6):
    """
    Simulate a basic binding experiment.
    """
     
    scalar = 20**(sequence_length)
    library_size = scalar*2.8e-5
    
    print(scalar, library_size, 9.77e-10*scalar)
    
    pipet = PipetteSampler()
    screen = BindingSampler(conc_constant=1.5e18)
    amplify = PhageAmplificationSampler()

    pool = Pool(sequence_length=sequence_length,alphabet=AMINO_ACIDS)
    pool.createUniformPool(library_size,affinity_max)

    try:
        print(0,pool.current_counts.size,sum(pool.current_counts))
        for i in range(num_rounds):
            pipet.runExperiment(pool,library_size)  #7e5) #100*scalar)
            screen.runExperiment(pool,1.5e-3*scalar) #4e7) #1*scalar)
            amplify.runExperiment(pool,6.8e-3*scalar,checkpoint=True) #1.75e8) #100*scalar,checkpoint=True)

            print(i+1,pool.current_counts.size,sum(pool.current_counts))
    
    except PoolIsEmptyError:
        pass
    
    
    return pool






In [ ]:
import sys
import pool, samplers, utility
from scipy.optimize import minimize

class Exp:
    """
    """
    
    def __init__(self,sequence_length=8,alphabet=pool.AMINO_ACIDS):
        """
        
        """
        
        # These are empirically-derived numbers that indicate the fraction of 
        # alphabet_size**sequence_length possible sequences pass through each
        # simulation step.
        self.company_tube_fx =      2.60e-3      # number of sequences in tube from company
        self.pipet_onto_plate_fx =  2.81e-5      # number of sequences pipetted into well
        self.protein_on_plate_fx =  1.50e-3      # number of protein molecules in well
        self.amplify_post_bind_fx = 6.40e-3      # number of phage after amplifying
        self.illumina_reads_fx    = 2.44e-10     # number of illumina reads per round
        self.first_round_recov_fx = 9.77e-10     # number of phage recovered after binding

        # Calculate number of possible sequences
        self.sequence_length = sequence_length
        self.alphabet = alphabet
        self.total_possible = len(alphabet)**sequence_length
        
        # Determine the number of sequences in each 
        self.company_tube = int(self.company_tube_fx*self.total_possible)
        self.pipet_onto_plate = int(self.pipet_onto_plate_fx*self.total_possible)
        self.protein_on_plate = int(self.protein_on_plate_fx*self.total_possible)
        self.amplify_post_bind = int(self.amplify_post_bind_fx*self.total_possible)
        self.illumina_reads = int(self.illumina_reads_fx*self.total_possible)
        self.first_round_recov = int(self.first_round_recov_fx*self.total_possible)

    def create(self,affinity_max=1e6,skew=3,conc_const_guess=10.,specified_conc_const=None):
        """
        Create a simulation engine with a concentration constant that reproduces the
        observed experimental value.
        """
        
        # Create pipet sampler and amplifier.
        self.pipet = samplers.PipetteSampler()
        self.amplify = samplers.PhageAmplificationSampler()

        # Create a pool
        self.p = pool.Pool(sequence_length=self.sequence_length,alphabet=self.alphabet)
        self.p.createSkewedPool(self.pipet_onto_plate,affinity_max,scale_reps=skew)
        
        if specified_conc_const:
            self.conc_const = specified_conc_const
        else:
            # Optimize the concentration constant so, given the pool size, we reproduce what
            # we saw experimentally recovered on our first round.
            print("Optimizing concentration constant")
            conc_constant = minimize(self._conc_const_objective,conc_const_guess,
                                     method='nelder-mead',
                                     options={'xtol': 1, 'disp': False})
        
            print("Done")
            self.conc_const = conc_constant.x
        
        #Create a binding sampler
        self.screen = samplers.BindingSampler(conc_constant=10**self.conc_const)
        
        #Create an illumina sampler
        self.illumina = samplers.IlluminaSampler()
        

    def _conc_const_objective(self,conc_const):
        """
        """
                                                    
        screen = samplers.BindingSampler(conc_constant=(10**conc_const))
        self.pipet.runExperiment(self.p,self.pipet_onto_plate)
        recovery = screen.runExperiment(self.p,self.protein_on_plate,
                                        return_only_sample_size=True)
        self.p.reset()
            
        print(conc_const[0],recovery,"vs",self.first_round_recov)
            
        return abs(self.first_round_recov - recovery)

    def run(self,num_rounds=3):
        """
        """

        # Do sampling runs for num_rounds or until there are no phage left.
        try:
            print("{:10s}{:15s}{:15s}".format("round","num_unique","total_counts"))
            print("{:10d}{:15d}{:15d}".format(0,self.p.current_counts.size,sum(self.p.current_counts)))
            
            for i in range(num_rounds):
                self.pipet.runExperiment(self.p,self.pipet_onto_plate)
                self.screen.runExperiment(self.p,self.protein_on_plate)
                self.amplify.runExperiment(self.p,self.amplify_post_bind,checkpoint=True)
            
                print("{:10d}{:15d}{:15d}".format(i+1,self.p.current_counts.size,sum(self.p.current_counts)))
    
        except samplers.PoolIsEmptyError:
            pass

        # Input pool summary
        self.input_pool_dict = {}
        seqs = [self.p.mapper.intToSeq(s) for s in self.p.all_seq]
        affin = self.p.all_affinities
        initial_counts = self.p.round_counts(0)
        for i, s in enumerate(seqs):
            self.input_pool_dict[s] = (affin[i],initial_counts[i])
        
        # Result of sampling protocol, no illumina sampling
        self.actual_out_dict = self.p.round_count_dict
        
        # --------------------------------------------------------------------
        # Sample from the runs via simulated illumina sequencing of each round
        # --------------------------------------------------------------------
        
        self.illumina_out_dict = {}
        
        # List of illumina samples to take
        to_take = [i for i,s in enumerate(self.p.checkpoints) if s]       
        
        # Sample the illumina counts at each round
        samples = []
        for t in to_take:
            contents, counts = self.illumina.runExperiment(self.p,
                                                           sample_size=self.illumina_reads,
                                                           round_to_sample=t)
            samples.append(list(zip(contents,counts)))
        
        # Create a list of unique sequences
        unique_seqs = []
        for s in samples:
            unique_seqs.extend([k[0] for k in s])
        unique_seqs = list(dict([(s,()) for s in unique_seqs]).keys())
        
        # Create a dictionary that will store sequence mapped to counts at each round
        template  = [0 for i in range(len(samples))]
        out_dict = {}
        for u in unique_seqs:
            out_dict[u] = template[:]
        
        # Append counts at each round for each sequence
        for i, s in enumerate(samples):
            for c in s:
                out_dict[c[0]][i] = c[1]

        # Convert the sequences to pretty, actual sequences
        for k in out_dict:
            self.illumina_out_dict[self.p.mapper.intToSeq(k)] = out_dict[k][:]
            

    
e = Exp(sequence_length=7)
e.create(skew=3)
e.run()
        

Optimizing concentration constant
10.0 3743 vs 1
10.5 1185 vs 1
11.0 375 vs 1
11.5 119 vs 1
12.5 12 vs 1
13.5 1 vs 1
15.5 0 vs 1
14.5 0 vs 1
12.5 12 vs 1
14.0 0 vs 1
14.0 0 vs 1
13.0 4 vs 1
13.75 1 vs 1
Done
round     num_unique     total_counts   
         0          35967          35968
# Number of proteins taken 1
         1              1        8192000
# Number of proteins taken 6
         2              1        8192000
# Number of proteins taken 6


In [ ]:
p = runMe(8,num_rounds=3,affinity_max=1e6)


import pickle
f = open("yo.pickle","wb")
pickle.dump(p,f)
#pickle.dump(p.round_count_dict,f)
f.close()

def runMe(sequence_length,num_rounds=3,affinity_max=1e6,alphabet=pool.AMINO_ACIDS,
          pipet_scalar=2.81e-5,screen_scalar=1.5e-3,amplify_scalar=6.4e-3,company_scalar=2.6e-3,
          conc_constant=5e14):
    """
    Simulate a basic binding experiment.  
    """
    
    total_possible = len(alphabet)**(sequence_length)
    library_size = total_possible*company_scalar
    
    pipet_scale = pipet_scalar*total_possible
    protein_on_plate_scale = screen_scalar*total_possible
    amplify_scale = amplify_scalar*total_possible
    
    self.pipet = samplers.PipetteSampler()
    self.screen = samplers.BindingSampler(conc_constant=conc_constant)
    self.amplify = samplers.PhageAmplificationSampler()

    p = pool.Pool(sequence_length=sequence_length,alphabet=alphabet)
    p.createScaledPool(pipet_scale,affinity_max,scale_reps=3)

    
    samples = []
    
    contents, counts = amplify.runExperiment(p,650,append_to_current=False)
    samples.append(list(zip(contents,counts)))
    
    try:
        print(0,p.current_counts.size,sum(p.current_counts))
        sys.stdout.flush()
        for i in range(num_rounds):
            self.pipet.runExperiment(p,pipet_scale)
            self.screen.runExperiment(p,protein_on_plate_scale)
            self.amplify.runExperiment(p,amplify_scale,checkpoint=True)
            
            self.contents, counts = amplify.runExperiment(p,650,append_to_current=False)
            samples.append(list(zip(contents,counts)))

            print(i+1,p.current_counts.size,sum(p.current_counts))
    
    except samplers.PoolIsEmptyError:
        pass

    
    unique_keys = []
    for s in samples:
        unique_keys.extend([k[0] for k in s])
    
    unique_keys = list(dict([(k,()) for k in unique_keys]).keys())
    template  = [0 for i in range(len(samples))]
    out_dict = {}
    for u in unique_keys:
        out_dict[u] = template[:]
        
    for i, s in enumerate(samples):
        for c in s:
            out_dict[c[0]][i] = c[1]
        
    return out_dict
    
    
    #return p


import matplotlib.pyplot as plt
%matplotlib inline

pool.round_affinities(0)

m = len(pool.checkpoints)
for i, x in enumerate(pool.checkpoints):
    if x == True:
        plt.plot(pool.round_affinities(i),pool.round_counts(i)/np.sum(pool.round_counts(i)),"o",color=[1-i/m,1-i/m,i/m])


unique_contents = []
for i, x in enumerate(pool.checkpoints):
    if x == True:
        unique_contents.extend(pool.round_contents(i))
        
counter = 0
out_dict = dict([(k,[0,0,0,0,0]) for k in unique_contents])
for i, x in enumerate(pool.checkpoints):
    if x == True:        
        counts = pool.round_counts(i)
        affinities = pool.round_affinities(i)
        
        for j, x in enumerate(pool.round_contents(i)):            
            out_dict[x][counter] = counts[j]
            out_dict[x][4] = affinities[j]
        
        counter = counter + 1
    
final_dict = {}
for k in out_dict.keys():
    if out_dict[k][1] != 0:
        final_dict[k] = out_dict[k][:]

plt.xscale('log')
plt.yscale('log')
plt.show()
    

out = []
for k in final_dict.keys():
    x = final_dict[k]
    out.append((final_dict[k][4],final_dict[k]))

out.sort()

for a in out:
    plt.plot([1,2,3],a[1][1:4])
    #plt.plot([1,2,3],a[1][1:4],"o")
    
plt.show()